In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm, trange
from math import sqrt
import pandas as pd
import numpy as np
import heapq
import random
import math

In [2]:
n,m=943,1682
cols = ['user_id', 'item_id', 'rating', 'ts']
base = pd.read_csv('../../ml-100k/u1.base',sep='\t',names=cols)
test = pd.read_csv('../../ml-100k/u1.test',sep='\t',names=cols)

In [3]:
# 创建uesr-item矩阵
train_matrix = np.zeros((n, m))
test_matrix = np.zeros((n, m))
K = 50
y_ui = np.zeros((n, m))
for i in base.itertuples():
    if i[3]>=4:
        train_matrix[i[1] - 1, i[2] - 1] = i[3]

for i in test.itertuples():
    if i[3]>=4:
        test_matrix[i[1] - 1, i[2] - 1] = i[3]

# 创建y矩阵
for i in range(n):
    for j in range(m):
        if train_matrix[i,j] >= 4:
            y_ui[i,j] = 1
# 确定物品集与用户集
I = list(np.where(sum(train_matrix+test_matrix)!=0)[0])
U = list(np.where(sum(train_matrix.T+test_matrix.T)!=0)[0])

In [4]:
# Item-Based OCCF
# Jaccard Index
skj = np.zeros((m,m))
for i in tqdm(I):
    for j in I:
        if i!=j:
            uk = len(set(np.where(train_matrix[:,i]!=0)[0]).union(set(np.where(train_matrix[:,j]!=0)[0])))
            if uk != 0:
                skj[i,j] = len(set(np.where(train_matrix[:,i]!=0)[0]).intersection(set(np.where(train_matrix[:,j]!=0)[0])))/uk  
IOCCF = np.zeros((n,m))
for i in tqdm(U):
    for j in I:
        idx = list(set(np.where(train_matrix[i,:]!=0)[0]).intersection(set(np.where(skj[j,:]!=0)[0])))
        ls = skj[j,idx]
        ls = sorted(ls,reverse=True)
        IOCCF[i,j] = sum(ls[:K])
rank_ioccf = list()
for i in tqdm(U):
    exclude = list(np.where(train_matrix[i,:]!=0)[0])
    ls = list()
    for j in I:
        if j in exclude:
            continue
        ls.append((j,IOCCF[i,j]))
    ls = sorted(ls, key=lambda x: x[1], reverse=True)
    rank_ioccf.append([ls[j][0] for j in range(len(ls))])

100%|███████████████████████████████████████████████████████████████████████████████| 942/942 [00:02<00:00, 392.89it/s]


In [5]:
pd.DataFrame(skj)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.000000,0.075862,0.056338,0.127273,0.055556,0.014388,0.257294,0.179641,0.171123,0.062500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.075862,0.000000,0.014925,0.102362,0.125000,0.000000,0.075556,0.094595,0.035176,0.045977,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.056338,0.014925,0.000000,0.049180,0.016949,0.027027,0.064815,0.027397,0.065934,0.025974,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.127273,0.102362,0.049180,0.000000,0.064516,0.009174,0.148289,0.187166,0.122363,0.063380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.055556,0.125000,0.016949,0.064516,0.000000,0.000000,0.049327,0.062069,0.042105,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# User-Based OCCF
# Jaccard Index
swu = np.zeros((n,n))
Ut = list(np.where(sum(train_matrix.T)!=0)[0])
for i in tqdm(Ut):
    for j in Ut:
        if i!=j:
            uk = len(set(np.where(train_matrix[i,:]!=0)[0]).union(set(np.where(train_matrix[j,:]!=0)[0])))
            if uk != 0:
                swu[i,j] = len(set(np.where(train_matrix[i,:]!=0)[0]).intersection(set(np.where(train_matrix[j,:]!=0)[0])))/uk
UOCCF = np.zeros((n,m))
for j in tqdm(I):
    for i in Ut:
        idx = list(set(np.where(train_matrix[:,j]!=0)[0]).intersection(set(np.where(swu[i,:]!=0)[0])))
        ls = swu[i,idx]
        ls = sorted(ls,reverse=True)
        UOCCF[i,j] = sum(ls[:K])
rank_uoccf = list()
for i in tqdm(Ut):
    exclude = list(np.where(train_matrix[i,:]!=0)[0])
    ls = list()
    for j in I:
        if j in exclude:
            continue
        ls.append((j,UOCCF[i,j]))
    ls = sorted(ls, key=lambda x: x[1], reverse=True)
    rank_uoccf.append([ls[j][0] for j in range(len(ls))])

100%|███████████████████████████████████████████████████████████████████████████████| 942/942 [00:02<00:00, 339.96it/s]


In [6]:
# Hybrid
r_hybrid = 0.5
HOCCF = r_hybrid*IOCCF+(1-r_hybrid)*UOCCF
rank_hoccf = list()
for i in tqdm(U):
    exclude = list(np.where(train_matrix[i,:]!=0)[0])
    ls = list()
    for j in I:
        if j in exclude:
            continue
        ls.append((j,HOCCF[i,j]))
    ls = sorted(ls, key=lambda x: x[1], reverse=True)
    rank_hoccf.append([ls[j][0] for j in range(len(ls))])

100%|███████████████████████████████████████████████████████████████████████████████| 942/942 [00:02<00:00, 382.30it/s]


In [10]:
def PreK(rank, truth, k):
    prek = 0
    ure = 0
    for i in U:
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        preuk = 0
        ure+=1
        for j in range(k):
            if rank[i][j] in ls:
                preuk+=1
        prek+=preuk/k
    return prek/ure

In [11]:
def RecK(rank, truth, k):
    reck = 0
    ure = 0
    for i in U:
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        recuk = 0
        ure += 1
        for j in range(k):
            if rank[i][j] in ls:
                recuk += 1
        reck+=recuk/ts
    return reck/ure

In [12]:
print("Item-based Pre@5:"+str(PreK(rank_ioccf, test_matrix, 5)))
print("Item-based Rec@5:"+str(RecK(rank_ioccf, test_matrix, 5)))
print("User-based Pre@5:"+str(PreK(rank_uoccf, test_matrix, 5)))
print("User-based Rec@5:"+str(RecK(rank_uoccf, test_matrix, 5)))
print("Hybrid Pre@5:"+str(PreK(rank_hoccf, test_matrix, 5)))
print("Hybrid Rec@5:"+str(RecK(rank_hoccf, test_matrix, 5)))

Item-based Pre@5:0.3530701754385959
Item-based Rec@5:0.11469222808336291
User-based Pre@5:0.3916666666666662
User-based Rec@5:0.12628445234304111
Hybrid Pre@5:0.38508771929824487
Hybrid Rec@5:0.12863426765771127
